# Lab 02: Implement Quality Gates with Pandera

Goal: codify 6–10 checks as a Pandera schema, run it, and see clear messages.


## 1) Install Pandera (Colab friendly)
- Do: Run the pip install.
- Why: Pandera powers declarative checks; one-line install keeps the session light.
- You should see: Installation logs ending without errors.
- If it doesn't look right: Rerun the cell; ensure internet is available in Colab.


In [ ]:
!pip -q install pandera[pandas]


## 2) Load data and imports
- Do: Load the cleaned CSV and import Pandera.
- Why: Schema depends on column names and types; loading first avoids mismatches.
- You should see: Dataframe preview.
- If it doesn't look right: Check the path; ensure headers are correct.


In [ ]:
from pathlib import Path

import pandas as pd
import pandera as pa
from pandera import Check, Column, DataFrameSchema

# Prefer the cleaned CSV from Lab 01; fall back to this lab's inputs; allow Colab upload.
candidates = [
    Path('../lab-01/inputs/collection_cleaned.csv'),
    Path('../../lab-01/inputs/collection_cleaned.csv'),
    Path('../inputs/collection_cleaned.csv'),
    Path('inputs/collection_cleaned.csv'),
    Path('collection_cleaned.csv'),
    Path('/content/collection_cleaned.csv'),
]

csv_path = next((p for p in candidates if p.exists()), None)
if csv_path is None:
    raise FileNotFoundError(
        "Couldn't find 'collection_cleaned.csv'.\n\n"
        "Provide the file, then either:\n"
        "- Put it in Lab 01 at '../lab-01/inputs/collection_cleaned.csv' (relative to lab-02/), or\n"
        "- Put it at '../inputs/collection_cleaned.csv' (relative to this notebook folder), or\n"
        "- Upload it in Colab so it appears as '/content/collection_cleaned.csv', or\n"
        "- Adjust the candidates list in this cell.\n\n"
        "Tried: " + ", ".join(str(p) for p in candidates)
    )

df = pd.read_csv(csv_path)

required_columns = {'id', 'title', 'creator', 'place', 'rights', 'date'}
missing = required_columns - set(df.columns)
if missing:
    raise ValueError(
        "CSV loaded, but it's missing expected columns: "
        + ", ".join(sorted(missing))
        + "\nFound columns: "
        + ", ".join(df.columns)
    )

df.head()


## 3) Define the schema
- Do: Create a DataFrameSchema with checks.
- Why: Makes rules of trust explicit and reusable.
- You should see: The schema object printed.
- If it doesn't look right: Ensure column names match the CSV; check for typos.


In [ ]:
schema = DataFrameSchema({
    'id': Column(pa.Int64, Check.greater_than(0), nullable=False, coerce=True),
    'title': Column(str, Check.str_length(min_value=1), nullable=False),
    'creator': Column(str, Check.str_length(min_value=1), nullable=False),
    'place': Column(str, Check.isin(['New York City', 'Albany']), nullable=False),
    'rights': Column(str, Check.isin(['Public Domain', 'CC BY 4.0', 'Rights Reserved']), nullable=False),
    'date': Column(pa.Int64, Check.between(1800, 2025), nullable=False, coerce=True),
}, coerce=True)

schema


## 4) Validate the dataframe
- Do: Run the schema against the dataframe.
- Why: Executes all checks and surfaces failures clearly.
- You should see: A validated dataframe (same as input) if all checks pass.
- If it doesn't look right: Read the error messages; they point to columns and offending values.


In [ ]:
validated = schema.validate(df)
validated.head()


## 5) Save schema for reuse
- Do: Save the schema code to `deliverables/validation_schema.py`.
- Why: Makes the gate portable and versionable for pipelines.
- You should see: A Python file on disk with your schema.
- If it doesn't look right: Check path spelling; ensure the deliverables folder exists.


In [ ]:
schema_code = """
import pandera as pa
from pandera import Column, Check, DataFrameSchema

schema = DataFrameSchema({
    'id': Column(pa.Int64, Check.greater_than(0), nullable=False, coerce=True),
    'title': Column(str, Check.str_length(min_value=1), nullable=False),
    'creator': Column(str, Check.str_length(min_value=1), nullable=False),
    'place': Column(str, Check.isin(['New York City', 'Albany']), nullable=False),
    'rights': Column(str, Check.isin(['Public Domain', 'CC BY 4.0', 'Rights Reserved']), nullable=False),
    'date': Column(pa.Int64, Check.between(1800, 2025), nullable=False, coerce=True),
})

"""
from pathlib import Path

Path('deliverables').mkdir(parents=True, exist_ok=True)

schema_path = Path('deliverables/validation_schema.py')
with open(schema_path,'w') as f:
    f.write(schema_code)
print(f'Saved schema to {schema_path.resolve()}')
